# Natural Language Processing Assignment: Spam Filter
## Import necessary libs and datasets

#### 데이터 가져오기

In [315]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
import urllib.request

In [316]:
import numpy as np
import pandas as pd
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/mohitgupta-omg/Kaggle-SMS-Spam-Collection-Dataset-/master/spam.csv", filename="spam.csv")
data = pd.read_csv('spam.csv', encoding='latin1')

import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

In [317]:
#spam이 1
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']

data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data['text'] = data['v2']
data['isSpam'] = data['v1']

del data['v1'], data['v2']

print(f'Data Shape: {data.shape}')
# imbalanced data
print(data['isSpam'].value_counts())

Data Shape: (5572, 2)
0    4825
1     747
Name: isSpam, dtype: int64


### 전처리


#### 특수기호, 영어 아닌 문자, 이중 space 제거, 대문자 소문자로 전환

In [318]:
import re

pattern1 = '[^\w\s]' #특수기호 제거
pattern2 = re.compile(r'\s+') #이중 space 제거
pattern3 = re.compile('[^a-zA-Z]') #영어가 아닌 문자 공백으로 대체 

def preprocess(content)   :
    content = re.sub(pattern1, ' ', content)
    content = re.sub(pattern3, ' ', content)
    content = re.sub(pattern2, ' ', content)
    content = content.lower() #대문자 소문자로 전환
        
    return content

In [319]:
data['text_n'] = data['text'].apply(lambda x: preprocess(x))

#### 불용어 제거

In [320]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [321]:
# is that a the 등 불용어 제거
from nltk.corpus import stopwords  
stopwords.words('english')[:10]  

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [322]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stop_words = set(stopwords.words('english')) 

def stop_word_remove(content):
    token = word_tokenize(content)
    result = []
    for w in token:
        if w not in stop_words:
            result.append(w)
    text = ' '.join(result)
    return text

In [323]:
data['text_n'] = data['text_n'].apply(lambda x: stop_word_remove(x))

#### glove pre-trained word vector 가져오기


In [324]:
import numpy as np
import gensim

In [325]:
cd /content/drive/My Drive/esc

/content/drive/My Drive/esc


In [329]:
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [330]:
print(word2vec_model.vectors.shape)

(3000000, 300)


In [333]:
'''
import os
embedding_dict = dict()
f = open(os.path.join('glove.6B.100d.txt'), encoding='utf-8')
for line in f:
    word_vector = line.split()
    word = word_vector[0]
    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환
    embedding_dict[word] = word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))
'''

"\nimport os\nembedding_dict = dict()\nf = open(os.path.join('glove.6B.100d.txt'), encoding='utf-8')\nfor line in f:\n    word_vector = line.split()\n    word = word_vector[0]\n    word_vector_arr = np.asarray(word_vector[1:], dtype='float32') # 100개의 값을 가지는 array로 변환\n    embedding_dict[word] = word_vector_arr\nf.close()\n\nprint('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))\n"

In [334]:
print(embedding_dict['respectable'])
print(len(embedding_dict['respectable']))
#임베딩 된 벡터의 차원은 100차원 

[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

#### pre-trained vector에 없는 단어 제거

In [335]:
#우리가 토큰화 한 단어 들 중 glove에서 학습된 단어 벡터가 무엇인지 알아보자
word_in_glove = []
unknown = []
for word in unique_word_list:
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        word_in_glove.append(word)
    if embedding_vector is None:
        unknown.append(word)

In [336]:
len(word_in_glove)

6235

In [341]:
word_in_glove

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat',
 'ok',
 'lar',
 'joking',
 'wif',
 'oni',
 'free',
 'entry',
 'comp',
 'win',
 'fa',
 'cup',
 'final',
 'tkts',
 'st',
 'may',
 'text',
 'receive',
 'question',
 'std',
 'txt',
 'rate',
 'apply',
 'dun',
 'say',
 'early',
 'hor',
 'already',
 'nah',
 'think',
 'goes',
 'usf',
 'lives',
 'around',
 'though',
 'hey',
 'darling',
 'week',
 'word',
 'back',
 'like',
 'fun',
 'still',
 'tb',
 'xxx',
 'send',
 'rcv',
 'even',
 'brother',
 'speak',
 'treat',
 'aids',
 'patent',
 'per',
 'request',
 'melle',
 'oru',
 'set',
 'callers',
 'press',
 'copy',
 'friends',
 'winner',
 'valued',
 'network',
 'customer',
 'selected',
 'prize',
 'reward',
 'claim',
 'call',
 'code',
 'kl',
 'valid',
 'hours',
 'mobile',
 'months',
 'entitled',
 'update',
 'latest',
 'colour',
 'mobiles',
 'camera',
 'co',
 'gon',
 'na',
 'home',
 'soon',
 'want',
 'talk',
 'stuff',
 'any

In [418]:
def remove_word(sen):
  word_list = []
  tok = word_tokenize(sen)
  for word in tok:
    if word in word_in_glove:
        word_list.append(word)
  new_sen = " ".join(word_list)
  return new_sen

In [421]:
data['text_n'] = data['text_n'].apply(lambda x: remove_word(x))

#### 문장 array 변환나 및 padding


In [422]:
text_list = []
for i in data.text_n:
  text_list.append(str(i))

In [423]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_list)
sequences = tokenizer.texts_to_sequences(text_list)

In [433]:
max_len=max(len(word_tokenize(l)) for l in data.text_n)
max_len

71

In [434]:
X = pad_sequences(sequences, maxlen = max_len)

In [435]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(data.isSpam)

y = idx_encode.transform(data.isSpam)

In [436]:
print(X[0])
print(y[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    8 3429  649  580  502 1074   49  219  802 2392 1075   10 3430
   57]
0


#### 임베딩 테이블 만들기

In [437]:
#우리가 토큰화 한 단어 들 중 glove에서 학습된 단어 벡터가 무엇인지 알아보자
word_in_glove = []
unknown = []
for word in unique_word_list:
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        word_in_glove.append(word)
    if embedding_vector is None:
        unknown.append(word)

In [438]:
len(word_in_glove)

6235

In [439]:
for ind, word in enumerate(word_in_glove):
  dict[word] = ind

In [440]:
dict

{'padding_idx': 0,
 'unk_idx': 1,
 'go': 0,
 'jurong': 1,
 'point': 2,
 'crazy': 3,
 'available': 4,
 'bugis': 5,
 'great': 6,
 'world': 7,
 'la': 8,
 'buffet': 9,
 'cine': 10,
 'got': 11,
 'amore': 12,
 'wat': 13,
 'ok': 14,
 'lar': 15,
 'joking': 16,
 'wif': 17,
 'oni': 18,
 'free': 19,
 'entry': 20,
 'comp': 21,
 'win': 22,
 'fa': 23,
 'cup': 24,
 'final': 25,
 'tkts': 26,
 'st': 27,
 'may': 28,
 'text': 29,
 'receive': 30,
 'question': 31,
 'std': 32,
 'txt': 33,
 'rate': 34,
 'apply': 35,
 'dun': 36,
 'say': 37,
 'early': 38,
 'hor': 39,
 'already': 40,
 'nah': 41,
 'think': 42,
 'goes': 43,
 'usf': 44,
 'lives': 45,
 'around': 46,
 'though': 47,
 'hey': 48,
 'darling': 49,
 'week': 50,
 'word': 51,
 'back': 52,
 'like': 53,
 'fun': 54,
 'still': 55,
 'tb': 56,
 'xxx': 57,
 'send': 58,
 'rcv': 59,
 'even': 60,
 'brother': 61,
 'speak': 62,
 'treat': 63,
 'aids': 64,
 'patent': 65,
 'per': 66,
 'request': 67,
 'melle': 68,
 'oru': 69,
 'set': 70,
 'callers': 71,
 'press': 72,
 'cop

In [441]:
embedding_dim = 100
vocab_size=6237
embedding_matrix = np.zeros((vocab_size, embedding_dim))

In [442]:
for word, i in dict.items():
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector 

In [443]:
embedding_matrix[0]

array([-0.078894  ,  0.46160001,  0.57779002, -0.71636999, -0.13121   ,
        0.41859999, -0.29155999,  0.52006   ,  0.089986  , -0.35062   ,
        0.51754999,  0.51998001,  0.15218   ,  0.41485   , -0.12377   ,
       -0.37222001,  0.0273    ,  0.75673002, -0.8739    ,  0.58934999,
        0.46662   ,  0.62918001,  0.092603  , -0.012868  , -0.015169  ,
        0.25567001, -0.43024999, -0.77667999,  0.71449   , -0.38339999,
       -0.69638002,  0.23522   ,  0.11396   ,  0.02778   ,  0.071357  ,
        0.87409002, -0.12809999,  0.063576  ,  0.067867  , -0.50181001,
       -0.28523001, -0.072536  , -0.50738001, -0.69139999, -0.53579003,
       -0.11361   , -0.38234001, -0.12414   ,  0.011214  , -1.16219997,
        0.037057  , -0.18494999,  0.01416   ,  0.87193   , -0.097309  ,
       -2.35649991, -0.14554   ,  0.28275001,  2.00530005,  0.23439001,
       -0.38297999,  0.69538999, -0.44916001, -0.094157  ,  0.90526998,
        0.65763998,  0.27627999,  0.30688   , -0.57780999, -0.22

#### train/test data 만들기


In [454]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,
                                                   stratify=y, test_size=0.1)

In [455]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5014, 71)
(5014,)
(558, 71)
(558,)


In [459]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3)

In [460]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(3509, 71)
(3509,)
(1505, 71)
(1505,)


### 모델 학습

In [461]:
label_idx={}
label_idx['ham']=0
label_idx['spam']=1
label_idx

{'ham': 0, 'spam': 1}

In [462]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate

In [463]:
filter_sizes = [2,3,5]
num_filters = 512
drop = 0.5

In [464]:
model_input = Input(shape = (max_len,))
z = Embedding(vocab_size, embedding_dim, weights=[embedding_matrix],
                      input_length=max_len, trainable=False)(model_input)

conv_blocks = []

for sz in filter_sizes:
    conv = Conv1D(filters = num_filters,
                         kernel_size = sz,
                         padding = "valid",
                         activation = "relu",
                         strides = 1)(z)
    conv = GlobalMaxPooling1D()(conv)
    conv = Flatten()(conv)
    conv_blocks.append(conv)

z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
z = Dropout(drop)(z)
model_output = Dense(len(label_idx)-1, activation='softmax')(z)

model = Model(model_input, model_output)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.summary()

Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            [(None, 71)]         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 71, 100)      623700      input_8[0][0]                    
__________________________________________________________________________________________________
conv1d_21 (Conv1D)              (None, 70, 512)      102912      embedding_7[0][0]                
__________________________________________________________________________________________________
conv1d_22 (Conv1D)              (None, 69, 512)      154112      embedding_7[0][0]                
_______________________________________________________________________________________

In [465]:
history = model.fit(X_train, y_train,
          batch_size=64,
          epochs=10,
          validation_data = (X_val, y_val))

Epoch 1/10
55/55 [==============================] - 1s 10ms/step - loss: 0.0000e+00 - acc: 0.1325 - val_loss: 0.0000e+00 - val_acc: 0.1375
Epoch 2/10
55/55 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - acc: 0.1325 - val_loss: 0.0000e+00 - val_acc: 0.1375
Epoch 3/10
55/55 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - acc: 0.1325 - val_loss: 0.0000e+00 - val_acc: 0.1375
Epoch 4/10
55/55 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - acc: 0.1325 - val_loss: 0.0000e+00 - val_acc: 0.1375
Epoch 5/10
55/55 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - acc: 0.1325 - val_loss: 0.0000e+00 - val_acc: 0.1375
Epoch 6/10
55/55 [==============================] - 0s 6ms/step - loss: 0.0000e+00 - acc: 0.1325 - val_loss: 0.0000e+00 - val_acc: 0.1375
Epoch 7/10
55/55 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - acc: 0.1325 - val_loss: 0.0000e+00 - val_acc: 0.1375
Epoch 8/10
55/55 [===============

### 평가

In [467]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.argmax(axis=-1) # 예측된 정수 시퀀스로 변환

In [468]:
y_predicted = idx_encode.inverse_transform(y_predicted) # 정수 시퀀스를 레이블에 해당하는 텍스트 시퀀스로 변환
y_test = idx_encode.inverse_transform(y_test) # 정수 시퀀스를 레이블에 해당하는 텍스트 시퀀스로 변환

In [469]:
print('accuracy: ', sum(y_predicted == y_test) / len(y_test))
print("Precision, Recall and F1-Score:\n\n", classification_report(y_test, y_predicted))

accuracy:  0.8655913978494624
Precision, Recall and F1-Score:

               precision    recall  f1-score   support

           0       0.87      1.00      0.93       483
           1       0.00      0.00      0.00        75

    accuracy                           0.87       558
   macro avg       0.43      0.50      0.46       558
weighted avg       0.75      0.87      0.80       558



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
